1) Import libraries

In [1]:
import pandas as pd

2) Import the layoff data

In [2]:
# Read the CSV file for New York layoffs
df_ny = pd.read_csv('../data/tech_layoffs_ny.csv')
# Display the first few rows
print("New York Layoffs:") 
df_ny.head()


New York Layoffs:


,State,Company,City,Number of Workers,WARN Received Date,Effective Date,Closure/Layoff,Temporary/Permanent,Union,Region,County,Industry,Notes
0,New York,Victoria's Secret (5th Avenue store & Herald S...,New York,740.0,08/31/2020,04/05/2020,Temporary Plant Closing,NaN,NaN,New York City,New York,Retail store,Unforeseeable business circumstances prompted ...
1,New York,"Tapestry, Inc.",New York,290.0,08/31/2020,03/13/2020,Temporary Plant Closing/Plant Layoff,NaN,NaN,New York City,New York,American multinational luxury fashion holding ...,Unforeseeable business circumstances prompted ...
2,New York,"Galaxy Restaurants Catering Group, LP aka Cons...",Bronx,210.0,08/31/2020,03/16/2020,Temporary Plant Closing/Plant Layoff,NaN,NaN,New York City,Bronx,Restaurant,Unforeseeable business circumstances prompted ...
3,New York,Access - Supports for Living Inc. (at U.S. Mer...,Kings Point,53.0,08/28/2020,11/30/2020,Plant Closing,NaN,NaN,Long Island,Nassau,Service academy in Nassau County,Termination of Access’s service contract with ...
4,New York,Aramark (at Pricewaterhouse Coopers),New York,35.0,08/28/2020,03/25/2020,Temporary Plant Closing/Plant Layoff,NaN,NaN,New York City,New York,Food Services,Unforeseeable business circumstances prompted ...


In [3]:
# Read the CSV file for Texas layoffs
df_te = pd.read_csv('../data/tech_layoffs_te.csv')
# Display the first few rows
print("Texas Layoffs:")
df_te.head()

Texas Layoffs:


,State,Company,City,Number of Workers,WARN Received Date,Effective Date,Closure/Layoff,Temporary/Permanent,Union,Region,County,Industry,Notes
0,Texas,XiFin,Austin,87,06/04/2025,05/29/2025,NaN,NaN,NaN,Capital Area WDA,Travis,NaN,NaN
1,Texas,Eckerd Connects (David Carrasco Job Corps),El Paso,77,06/04/2025,06/13/2025,NaN,NaN,NaN,Borderplexjobs WDA,El Paso,NaN,NaN
2,Texas,Sunnova Energy Corporation,Houston,NaN,06/03/2025,05/30/2025,NaN,NaN,NaN,Gulf Coast WDA,Harris,NaN,NaN
3,Texas,JC Penney (Allaince Supply Chain),Haslet,296,06/03/2025,08/01/2025,NaN,NaN,NaN,Tarrant County WDA,Tarrant,NaN,NaN
4,Texas,Texana Center (Building A) Airport Service Center,Rosenberg,27,05/28/2025,05/21/2025,NaN,NaN,NaN,Gulf Coast WDA,Fort Bend,NaN,NaN


3) Other way to get the layoffs for both states, in all the big tech companies

In [4]:
df_amazon = pd.read_csv('../data/data_amazon.csv')
df_apple = pd.read_csv('../data/data_apple.csv')
df_facebook = pd.read_csv('../data/data_facebook.csv')
df_google = pd.read_csv('../data/data_google.csv')
df_microsoft = pd.read_csv('../data/data_microsoft.csv')
df_tesla = pd.read_csv('../data/data_tesla.csv')
df_twitter = pd.read_csv('../data/data_twitter.csv')
df_amazon.head()

,State,Company,City,Number of Workers,WARN Received Date,Effective Date,Closure/Layoff,Temporary/Permanent,Union,Region,County,Industry,Notes
0,California,Amazon (SJC11 Facility),Sunnyvale,33,05/15/2025,07/07/2025,Layoff Permanent,NaN,NaN,NaN,Santa Clara County,44-45 Retail Trade,1100 Enterprise Way Sunnyvale CA 94089
1,California,Amazon (SJC14 Facility),Sunnyvale,6,05/15/2025,07/07/2025,Layoff Permanent,NaN,NaN,NaN,Santa Clara County,44-45 Retail Trade,905 11th Avenue Sunnyvale CA 94089
2,California,Amazon (SJC31 Facility),Sunnyvale,5,05/15/2025,07/07/2025,Layoff Permanent,NaN,NaN,NaN,Santa Clara County,44-45 Retail Trade,1100 Discovery Way Sunnyvale CA 94089
3,California,Amazon (SJC32 Facility),Sunnyvale,8,05/15/2025,07/07/2025,Layoff Permanent,NaN,NaN,NaN,Santa Clara County,44-45 Retail Trade,1140 Enterprise Way Sunnyvale CA 94089
4,Virginia,Amazon MAK7,Manassas,88,03/12/2025,05/13/2025,Closure,NaN,NaN,Northern,NaN,NaN,NaN


In [5]:
# Combine all the dataframes into a single dataframe
df_combined = pd.concat([df_amazon, df_apple, df_facebook, df_google, df_microsoft, df_tesla, df_twitter], ignore_index=True)
df_combined.head()

,State,Company,City,Number of Workers,WARN Received Date,Effective Date,Closure/Layoff,Temporary/Permanent,Union,Region,County,Industry,Notes,Received Date,Effective\n Date,Layoff/Closure
0,California,Amazon (SJC11 Facility),Sunnyvale,33,05/15/2025,07/07/2025,Layoff Permanent,NaN,NaN,NaN,Santa Clara County,44-45 Retail Trade,1100 Enterprise Way Sunnyvale CA 94089,NaN,NaN,NaN
1,California,Amazon (SJC14 Facility),Sunnyvale,6,05/15/2025,07/07/2025,Layoff Permanent,NaN,NaN,NaN,Santa Clara County,44-45 Retail Trade,905 11th Avenue Sunnyvale CA 94089,NaN,NaN,NaN
2,California,Amazon (SJC31 Facility),Sunnyvale,5,05/15/2025,07/07/2025,Layoff Permanent,NaN,NaN,NaN,Santa Clara County,44-45 Retail Trade,1100 Discovery Way Sunnyvale CA 94089,NaN,NaN,NaN
3,California,Amazon (SJC32 Facility),Sunnyvale,8,05/15/2025,07/07/2025,Layoff Permanent,NaN,NaN,NaN,Santa Clara County,44-45 Retail Trade,1140 Enterprise Way Sunnyvale CA 94089,NaN,NaN,NaN
4,Virginia,Amazon MAK7,Manassas,88,03/12/2025,05/13/2025,Closure,NaN,NaN,Northern,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Keep only the layoffs from Texas
df_texas = df_combined[(df_combined['State'] == 'Texas')].copy()
# Display the first few rows of the dataframe
print("Combined Layoffs Data (Texas):")
df_texas.head()

Combined Layoffs Data (Texas):


,State,Company,City,Number of Workers,WARN Received Date,Effective Date,Closure/Layoff,Temporary/Permanent,Union,Region,County,Industry,Notes,Received Date,Effective\n Date,Layoff/Closure
29,Texas,Whole Food Market (Midtown Houston),Houston,110,11/08/2023,12/05/2023,NaN,NaN,NaN,Gulf Coast WDA,Harris,NaN,NaN,NaN,NaN,NaN
97,Texas,Whole Foods Market-Bolm Rd.,Austin,40,03/02/2017,04/30/2017,NaN,NaN,NaN,Capital Area WDA,Travis,NaN,NaN,NaN,NaN,NaN
103,Texas,Whole Foods Market-Houston,Houston,100,05/01/2015,07/03/2015,NaN,NaN,NaN,Gulf Coast WDA,Harris,NaN,NaN,NaN,NaN,NaN
106,Texas,Whole Foods Market - Dallas Bakehouse,Dallas,70,01/02/2012,03/11/2012,NaN,NaN,NaN,Dallas County WDA,Dallas,NaN,NaN,NaN,NaN,NaN
108,Texas,Whole Foods Market - Southwest Distribution Ce...,Austin,60,07/27/2010,09/26/2010,NaN,NaN,NaN,Capital Area WDB,Travis,NaN,NaN,NaN,NaN,NaN


In [7]:
# Keep only the layoffs from new York
df_ny = df_combined[(df_combined['State'] == 'New York')].copy()
# Display the first few rows of the dataframe
print("Combined Layoffs Data (New York):")
df_ny.head()

Combined Layoffs Data (New York):


,State,Company,City,Number of Workers,WARN Received Date,Effective Date,Closure/Layoff,Temporary/Permanent,Union,Region,County,Industry,Notes,Received Date,Effective\n Date,Layoff/Closure
83,New York,Instacart (at Whole Foods Market) Bedford Avenue,Brooklyn,43,03/13/2019,05/11/2019,Plant Closing,NaN,NaN,New York City,Kings,Food Services/Delivery,Economic,NaN,NaN,NaN
84,New York,Instacart (at Whole Foods Market) Greenwich St...,New York,26,03/13/2019,05/11/2019,Plant Closing,NaN,NaN,New York City,"New York,",Food Services/Delivery,Economic,NaN,NaN,NaN
85,New York,Instacart (at Whole Foods Market) Columbus Circle,New York,32,03/13/2019,05/11/2019,Plant Closing,NaN,NaN,New York City,"New York,",Food Services/Delivery,Economic,NaN,NaN,NaN
86,New York,Instacart (at Whole Foods Market) (95 East Hou...,New York,21,03/13/2019,05/11/2019,Plant Closing,NaN,NaN,New York City,"New York,",Food Services/Delivery,Economic,NaN,NaN,NaN
87,New York,Instacart (at Whole Foods Market) (4 Union Squ...,New York,21,03/13/2019,05/11/2019,Plant Closing,NaN,NaN,New York City,"New York,",Food Services/Delivery,Economic,NaN,NaN,NaN


In [9]:
# Keep only the layoffs from California
df_california = df_combined[(df_combined['State'] == 'California')].copy()
# Display the first few rows of the dataframe
print("Combined Layoffs Data (California):")
df_california.head()

Combined Layoffs Data (California):


,State,Company,City,Number of Workers,WARN Received Date,Effective Date,Closure/Layoff,Temporary/Permanent,Union,Region,County,Industry,Notes,Received Date,Effective\n Date,Layoff/Closure
0,California,Amazon (SJC11 Facility),Sunnyvale,33,05/15/2025,07/07/2025,Layoff Permanent,NaN,NaN,NaN,Santa Clara County,44-45 Retail Trade,1100 Enterprise Way Sunnyvale CA 94089,NaN,NaN,NaN
1,California,Amazon (SJC14 Facility),Sunnyvale,6,05/15/2025,07/07/2025,Layoff Permanent,NaN,NaN,NaN,Santa Clara County,44-45 Retail Trade,905 11th Avenue Sunnyvale CA 94089,NaN,NaN,NaN
2,California,Amazon (SJC31 Facility),Sunnyvale,5,05/15/2025,07/07/2025,Layoff Permanent,NaN,NaN,NaN,Santa Clara County,44-45 Retail Trade,1100 Discovery Way Sunnyvale CA 94089,NaN,NaN,NaN
3,California,Amazon (SJC32 Facility),Sunnyvale,8,05/15/2025,07/07/2025,Layoff Permanent,NaN,NaN,NaN,Santa Clara County,44-45 Retail Trade,1140 Enterprise Way Sunnyvale CA 94089,NaN,NaN,NaN
5,California,Amazon,Thousand Oaks,76,03/14/2025,05/13/2025,Closure Permanent,NaN,NaN,NaN,Ventura County,44-45 Retail Trade,140 Promenade Way Thousand Oaks CA 91362,NaN,NaN,NaN


In [10]:
# Keep only the layoffs from Washington
df_washington = df_combined[(df_combined['State'] == 'Washington')].copy()
# Display the first few rows of the dataframe
print("Combined Layoffs Data (Washington):")
df_washington.head()

Combined Layoffs Data (Washington):


,State,Company,City,Number of Workers,WARN Received Date,Effective Date,Closure/Layoff,Temporary/Permanent,Union,Region,County,Industry,Notes,Received Date,Effective\n Date,Layoff/Closure
14,Washington,Amazon,Tukwila,172,05/31/2024,08/01/2024,Closure,Permanent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57,Washington,Amazon,"Seattle, Bellevue","2,300",01/18/2023,03/19/2023,Layoff,Permanent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67,Washington,Amazon Health Services,Seattle,159,09/07/2022,12/01/2022,Closure,Permanent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82,Washington,Whole Foods Market,Vancouver,121,06/21/2019,08/18/2019,Closure,Permanent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
214,Washington,"Meta Platforms, Inc",Bellevue,307,11/11/2022,01/13/2023,Layoff,Permanent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


4) For each state, I want to know the number of workers who received the WARN per month

In [8]:
# Make sure the 'Date' column is in datetime format
df_texas['WARN Received Date'] = pd.to_datetime(df_texas['WARN Received Date'])
# Create a new column to just keep the month and year
df_texas['Month-Year'] = df_texas['WARN Received Date'].dt.to_period('M')
# Group by 'Month-Year' and count the number of layoffs
df_texas_monthly = df_texas.groupby('Month-Year')['Number of Workers'].sum().reset_index()
# Rename the columns for clarity
df_texas_monthly.columns = ['Month-Year', 'Total Layoffs in Texas']
# Display the monthly layoffs data for Texas
print("Monthly Layoffs Data for Texas:")
print(df_texas_monthly)

Monthly Layoffs Data for Texas:
  Month-Year Total Layoffs in Texas
0    2008-04                     52
1    2010-07                     60
2    2012-01                     70
3    2015-05                    100
4    2017-03                     40
5    2022-11                    222
6    2023-11                    110
7    2024-04                  2,688
8    2025-05                     50


In [11]:
df_ny['WARN Received Date'] = pd.to_datetime(df_ny['WARN Received Date'])
df_ny['Month-Year'] = df_ny['WARN Received Date'].dt.to_period('M')
df_ny_monthly = df_ny.groupby('Month-Year')['Number of Workers'].sum().reset_index()
df_ny_monthly.columns = ['Month-Year', 'Total Layoffs in New York']
print("Monthly Layoffs Data for New York:")
print(df_ny_monthly)

Monthly Layoffs Data for New York:
  Month-Year Total Layoffs in New York
0    2009-05                        26
1    2011-07                       275
2    2012-12                        27
3    2014-07                        32
4    2014-08                        47
5    2014-09                         1
6    2016-04                       895
7    2019-03     432632212116202113211


In [12]:
df_california['WARN Received Date'] = pd.to_datetime(df_california['WARN Received Date'])
df_california['Month-Year'] = df_california['WARN Received Date'].dt.to_period('M')
df_california_monthly = df_california.groupby('Month-Year')['Number of Workers'].sum().reset_index()
df_california_monthly.columns = ['Month-Year', 'Total Layoffs in California']       
print("Monthly Layoffs Data for California:")
print(df_california_monthly)

TypeError: can only concatenate str (not "int") to str

In [13]:
df_washington['WARN Received Date'] = pd.to_datetime(df_washington['WARN Received Date'])
df_washington['Month-Year'] = df_washington['WARN Received Date'].dt.to_period('M')
df_washington_monthly = df_washington.groupby('Month-Year')['Number of Workers'].sum().reset_index()
df_washington_monthly.columns = ['Month-Year', 'Total Layoffs in Washington']
print("Monthly Layoffs Data for Washington:")
print(df_washington_monthly)

Monthly Layoffs Data for Washington:
   Month-Year Total Layoffs in Washington
0     2004-08                          67
1     2009-01                         872
2     2009-04                           2
3     2009-05                       1,200
4     2009-09                          27
5     2014-07                       1,351
6     2014-09                         747
7     2014-10                         635
8     2019-06                         121
9     2022-09                         159
10    2022-11                         934
11    2023-01                    2,300878
12    2023-02                         617
13    2023-03                      559689
14    2023-05                         158
15    2023-06                          22
16    2023-07                         276
17    2024-05                         172
18    2025-05                       1,985
19    2025-06                         305
